DAT405 Assignment 3

Theodor Jendle - 9901134339 - 

Carl Hjalmarsson - 9305198930 - 

In [ ]:
import nbdime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

prot_angles = pd.read_csv("assignment-3-data.cvs", index_col = False)
